In [4]:
import numpy as np
import pandas as pd
import os
import keras

2024-11-08 13:25:11.683228: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 13:25:11.697708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 13:25:11.713251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 13:25:11.718191: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 13:25:11.731185: I tensorflow/core/platform/cpu_feature_guar

In [5]:
current_path = os.getcwd()
os.chdir(current_path)
print(f"Working directory set to: {current_path}")

Working directory set to: /sfs/gpfs/tardis/home/rmd9ev/Documents/MSDS/DS6050/Project


In [6]:
import pandas as pd
import glob

# Get all CSV file paths in the specified directory
csv_files = glob.glob("/sfs/gpfs/tardis/home/rmd9ev/Documents/MSDS/DS6050/Project/final_data/*.csv")

csv_files_sorted = sorted(csv_files)

# Read and concatenate all CSV files into a single DataFrame
dataframes = [pd.read_csv(file) for file in csv_files_sorted]
combined_df = pd.concat(dataframes, ignore_index=True)

# Inspect the combined DataFrame
#combined_df.head()

In [8]:
X_data = combined_df[['insurance','race','marital_status','gender','anchor_age','value','amount','amountuom','label','sequence_num']]

In [9]:
data_encoded = pd.get_dummies(X_data, columns=['insurance','race','marital_status','gender','amountuom','label','value'])

In [65]:
#NaN values in amount
data_encoded.isna().sum()

anchor_age                      0
amount                      15856
sequence_num                    0
insurance_Medicaid              0
insurance_Medicare              0
                            ...  
value_SIMV/PSV/AutoFlow         0
value_SPONT                     0
value_Sensor Medic (HFO)        0
value_Standby                   0
value_VOL/AC                    0
Length: 104, dtype: int64

In [19]:
 data_encoded['amount'] = data_encoded['amount'].fillna(0)

In [21]:
data_encoded.isna().sum().sum()

0

In [22]:
from sklearn.model_selection import train_test_split

# Split into features (X) and target labels (y)
X = data_encoded.values  # Replace 'input_column' with your actual column name
X = X.astype(np.float32)
y = combined_df['Died'].values  # Replace 'target_column' with your actual target column
y = y.astype(np.int32)

# Split data into training, validation, and testing sets (e.g., 70% train, 15% val, 15% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Testing set size:", len(X_test))

Training set size: 850479
Validation set size: 182245
Testing set size: 182246


In [78]:
import tensorflow as tf

# Define a function to convert arrays to TensorFlow Datasets
def to_tf_dataset(X, y, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=len(X)).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

#def to_tf_dataset(X, y, batch_size=32):
#    dataset = tf.data.Dataset.from_tensor_slices((X, y))
 #   dataset = dataset.shuffle(buffer_size=len(X)).padded_batch(
#        batch_size,
#        padded_shapes=([104,], []),
#        padding_values=(0.0, 0)
#        ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#    return dataset

# Create TensorFlow Datasets
train_dataset = to_tf_dataset(X_train, y_train)
val_dataset = to_tf_dataset(X_val, y_val)
test_dataset = to_tf_dataset(X_test, y_test)

# Inspect the dataset
#for x, y in train_dataset.take(1):
#    print(x, y)

In [24]:
# Inspect the dataset
for x, y in train_dataset.take(1):
    print(x, y)

tf.Tensor(
[[  78.    80.  1286.  ...    0.     0.     0. ]
 [  74.   102.   612.  ...    0.     0.     0. ]
 [  66.    80.   392.  ...    0.     0.     0. ]
 ...
 [  65.    80.    38.  ...    0.     0.     0. ]
 [  80.     1.5  952.  ...    0.     0.     0. ]
 [  77.   103.    37.  ...    0.     0.     0. ]], shape=(32, 104), dtype=float32) tf.Tensor([0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0], shape=(32,), dtype=int32)


2024-11-08 13:29:50.703805: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [43]:
element_spec = train_dataset.element_spec
print(element_spec)
#train_dataset.tf.shape

(TensorSpec(shape=(None, 104), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [86]:
#model 1
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1,input_shape = (None,1))
])

optimizer = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(loss= keras.losses.BinaryCrossentropy(), 
              optimizer=optimizer,
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)]
             )
history = model.fit(train_dataset, epochs=2,
                    validation_data=(val_dataset))

Epoch 1/2
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 119s 4ms/step - binary_accuracy: 0.7499 - loss: 2.4055 - val_binary_accuracy: 0.7520 - val_loss: 0.5599
Epoch 2/2
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 119s 4ms/step - binary_accuracy: 0.7512 - loss: 0.5611 - val_binary_accuracy: 0.7525 - val_loss: 0.5596


In [88]:
loss, accuracy = model.evaluate(test_dataset)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


5696/5696 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - binary_accuracy: 0.7509 - loss: 0.5614
Loss:  0.5596147179603577
Accuracy:  0.7525377869606018


In [89]:
# model 2
# Build the model
model = keras.models.Sequential([
    keras.layers.Masking(mask_value=0.0, input_shape=(None, 1)),  # Masking layer to handle variable-length sequences
    keras.layers.SimpleRNN(32),  # SimpleRNN with 32 units
    keras.layers.Dense(1, activation='sigmoid')  # Example output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_dataset, epochs=2,
                    validation_data=(val_dataset))

Epoch 1/2
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 283s 11ms/step - accuracy: 0.7692 - loss: 0.4882 - val_accuracy: 0.7606 - val_loss: 0.4860
Epoch 2/2
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 282s 11ms/step - accuracy: 0.7718 - loss: 0.4842 - val_accuracy: 0.7861 - val_loss: 0.4799


In [90]:
loss, accuracy = model.evaluate(test_dataset)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


5696/5696 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.7874 - loss: 0.4788
Loss:  0.4783785045146942
Accuracy:  0.7868814468383789


In [ ]:
#imbalanced data?
#data augmentation?